In [ ]:
import numpy as np
import pandas as pd

from __future__ import print_function
#from sklearn.cross_validation import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [ ]:
df = pd.read_csv('/content/kddcup99_csv.csv')
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,lroot_shell,lsu_attempted,lnum_root,lnum_file_creations,lnum_shells,lnum_access_files,lnum_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Attack
0,0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal
1,0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal


In [ ]:
df.isna()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,lroot_shell,lsu_attempted,lnum_root,lnum_file_creations,lnum_shells,lnum_access_files,lnum_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Attack
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494015,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
494016,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
494017,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
494018,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
df = df.dropna()

## Coverting Categorical String into Numeric Data.
*   Protocol_Type
> tcp: 1, udp: 2, icmp: 3
*   Service
> private: 1, ftp_data: 2, eco_i: 3, telnet: 4, etc...
*   Flag
> REJ: 1, SF: 2, 	RSTO: 3, etc...

In [ ]:
# Prtotcol Type Conversion
df["protocol_type"]=df["protocol_type"].replace({"tcp":1})
df["protocol_type"]=df["protocol_type"].replace({"udp":2})
df["protocol_type"]=df["protocol_type"].replace({"icmp":3})

In [ ]:
# Creating new columns for numeric codes of Service and Flag
df['serv_code'] = pd.factorize(df['service'])[0] + 1
df['flag_code'] = pd.factorize(df['flag'])[0] + 1

In [ ]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,lroot_shell,lsu_attempted,lnum_root,lnum_file_creations,lnum_shells,lnum_access_files,lnum_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Attack,serv_code,flag_code
0,0,1,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal,1,1
1,0,1,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal,1,1
2,0,1,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal,1,1
3,0,1,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal,1,1
4,0,1,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal,1,1


In [ ]:
mapping = {'ipsweep': 'Probe','satan': 'Probe','nmap': 'Probe','portsweep': 'Probe','saint': 'Probe','mscan': 'Probe',
        'teardrop': 'DoS','pod': 'DoS','land': 'DoS','back': 'DoS','neptune': 'DoS','smurf': 'DoS','mailbomb': 'DoS',
        'udpstorm': 'DoS','apache2': 'DoS','processtable': 'DoS',
        'perl': 'U2R','loadmodule': 'U2R','rootkit': 'U2R','buffer_overflow': 'U2R','xterm': 'U2R','ps': 'U2R',
        'sqlattack': 'U2R','httptunnel': 'U2R',
        'ftp_write': 'R2L','phf': 'R2L','guess_passwd': 'R2L','warezmaster': 'R2L','warezclient': 'R2L','imap': 'R2L',
        'spy': 'R2L','multihop': 'R2L','named': 'R2L','snmpguess': 'R2L','worm': 'R2L','snmpgetattack': 'R2L',
        'xsnoop': 'R2L','xlock': 'R2L','sendmail': 'R2L',
        'normal': 'Normal'
        }

In [ ]:
df['attack_class'] = df['Attack'].apply(lambda v: mapping[v])

## Splitting the Dataset into Training and Testing Dataset 
Here, we have considered the ratio of 60:40 to split the dataset into train and test dataset respectively.

In [ ]:
training_data = df.sample(frac=0.6, random_state=25)
testing_data = df.drop(training_data.index)

In [ ]:
print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

No. of training examples: 296412
No. of testing examples: 197608


We can clearly observe that the dataset is splitted into train and test data with a ratio of 60:40

## Converting Attack Types into Multi Classification 0 and 1.


In [ ]:
df["attack_class"]=df["attack_class"].replace({"DoS":1})
df["attack_class"]=df["attack_class"].replace({"U2R":2})
df["attack_class"]=df["attack_class"].replace({"R2L":3})
df["attack_class"]=df["attack_class"].replace({"Probe":4})
df["attack_class"]=df["attack_class"].replace({"Normal":5})

In [ ]:
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,lroot_shell,lsu_attempted,lnum_root,lnum_file_creations,lnum_shells,lnum_access_files,lnum_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Attack,serv_code,flag_code,attack_class
0,0,1,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal,1,1,5
1,0,1,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal,1,1,5
2,0,1,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal,1,1,5
3,0,1,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal,1,1,5
4,0,1,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal,1,1,5


## Normalizing the Features of our Training Dataset

In [ ]:
import numpy as np
from sklearn import preprocessing
x = df.drop(["Attack", "service", "flag", "attack_class"], axis=1)
y= df["attack_class"]
y=y.astype('int')
normalized_arr = preprocessing.normalize(x)
print(normalized_arr)

[[0.00000000e+00 1.83384219e-04 3.31925437e-02 ... 0.00000000e+00
  1.83384219e-04 1.83384219e-04]
 [0.00000000e+00 1.84373540e-03 4.40652760e-01 ... 0.00000000e+00
  1.84373540e-03 1.84373540e-03]
 [0.00000000e+00 7.36287997e-04 1.73027679e-01 ... 0.00000000e+00
  7.36287997e-04 7.36287997e-04]
 ...
 [0.00000000e+00 8.04035169e-04 1.63219139e-01 ... 0.00000000e+00
  8.04035169e-04 8.04035169e-04]
 [0.00000000e+00 7.92909963e-04 2.30736799e-01 ... 0.00000000e+00
  7.92909963e-04 7.92909963e-04]
 [0.00000000e+00 7.81573245e-04 1.71164541e-01 ... 0.00000000e+00
  7.81573245e-04 7.81573245e-04]]


In [ ]:
norm_train_df=pd.DataFrame(normalized_arr,y)
norm_train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
attack_class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.000183,0.033193,0.999444,0.0,0.0,0.0,0.0,0.0,0.000183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001467,0.001467,0.0,0.0,0.0,0.0,0.000183,0.0,0.0,0.001650,0.001650,0.000183,0.0,0.000020,0.0,0.0,0.0,0.0,0.0,0.000183,0.000183
5,0.0,0.001844,0.440653,0.896055,0.0,0.0,0.0,0.0,0.0,0.001844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014750,0.014750,0.0,0.0,0.0,0.0,0.001844,0.0,0.0,0.035031,0.035031,0.001844,0.0,0.000092,0.0,0.0,0.0,0.0,0.0,0.001844,0.001844
5,0.0,0.000736,0.173028,0.984417,0.0,0.0,0.0,0.0,0.0,0.000736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005890,0.005890,0.0,0.0,0.0,0.0,0.000736,0.0,0.0,0.021352,0.021352,0.000736,0.0,0.000022,0.0,0.0,0.0,0.0,0.0,0.000736,0.000736
5,0.0,0.000737,0.161508,0.986011,0.0,0.0,0.0,0.0,0.0,0.000737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004425,0.004425,0.0,0.0,0.0,0.0,0.000737,0.0,0.0,0.028762,0.028762,0.000737,0.0,0.000022,0.0,0.0,0.0,0.0,0.0,0.000737,0.000737
5,0.0,0.000489,0.106126,0.993766,0.0,0.0,0.0,0.0,0.0,0.000489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002934,0.002934,0.0,0.0,0.0,0.0,0.000489,0.0,0.0,0.023964,0.023964,0.000489,0.0,0.000010,0.0,0.0,0.0,0.0,0.0,0.000489,0.000489


In [ ]:
norm_train_df.columns = x.columns
norm_train_df.head()

,duration,protocol_type,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,lnum_compromised,lroot_shell,lsu_attempted,lnum_root,lnum_file_creations,lnum_shells,lnum_access_files,lnum_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,serv_code,flag_code
attack_class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.000183,0.033193,0.999444,0.0,0.0,0.0,0.0,0.0,0.000183,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001467,0.001467,0.0,0.0,0.0,0.0,0.000183,0.0,0.0,0.001650,0.001650,0.000183,0.0,0.000020,0.0,0.0,0.0,0.0,0.0,0.000183,0.000183
5,0.0,0.001844,0.440653,0.896055,0.0,0.0,0.0,0.0,0.0,0.001844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014750,0.014750,0.0,0.0,0.0,0.0,0.001844,0.0,0.0,0.035031,0.035031,0.001844,0.0,0.000092,0.0,0.0,0.0,0.0,0.0,0.001844,0.001844
5,0.0,0.000736,0.173028,0.984417,0.0,0.0,0.0,0.0,0.0,0.000736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005890,0.005890,0.0,0.0,0.0,0.0,0.000736,0.0,0.0,0.021352,0.021352,0.000736,0.0,0.000022,0.0,0.0,0.0,0.0,0.0,0.000736,0.000736
5,0.0,0.000737,0.161508,0.986011,0.0,0.0,0.0,0.0,0.0,0.000737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004425,0.004425,0.0,0.0,0.0,0.0,0.000737,0.0,0.0,0.028762,0.028762,0.000737,0.0,0.000022,0.0,0.0,0.0,0.0,0.0,0.000737,0.000737
5,0.0,0.000489,0.106126,0.993766,0.0,0.0,0.0,0.0,0.0,0.000489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002934,0.002934,0.0,0.0,0.0,0.0,0.000489,0.0,0.0,0.023964,0.023964,0.000489,0.0,0.000010,0.0,0.0,0.0,0.0,0.0,0.000489,0.000489


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(normalized_arr,y,train_size=0.6,test_size=0.4, random_state=42)


## Feature Selection

In [ ]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier())

In [ ]:
sel.get_support()

array([False,  True, False,  True, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False,  True,  True, False, False, False, False,  True,
       False, False,  True,  True,  True, False, False, False, False,
       False, False, False,  True,  True])

In [ ]:
selected_feat= norm_train_df.columns[(sel.get_support())]
len(selected_feat)

11

In [ ]:
print(selected_feat)

Index(['protocol_type', 'dst_bytes', 'logged_in', 'count', 'srv_count',
       'same_srv_rate', 'dst_host_count', 'dst_host_srv_count',
       'dst_host_same_srv_rate', 'serv_code', 'flag_code'],
      dtype='object')


In [ ]:
X_train.shape

(296412, 11)

In [ ]:
X_train = X_train[:,sel.get_support()]
X_test = X_test[:,sel.get_support()]



In [ ]:
model = Sequential()
model.add(LSTM(4,input_dim=11))  # try using a GRU instead, for fun
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('sigmoid'))
#print(model.get_config())

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 4)                 256       
                                                                 
 dropout_1 (Dropout)         (None, 4)                 0         
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
 activation_1 (Activation)   (None, 1)                 0         
                                                                 
Total params: 261
Trainable params: 261
Non-trainable params: 0
_________________________________________________________________


In [ ]:
batch_size = 32

In [ ]:
# reshape input to be [samples, time steps, features]
X_train  = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger('training_set_iranalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, batch_size=batch_size, epochs=2, validation_data=(X_test, y_test),callbacks=[checkpointer,csv_logger])
model.save("lstm1layer_model.hdf5")

Epoch 1/10
9263/9263 [==============================] - 33s 3ms/step - loss: -16.3043 - accuracy: 0.7923 - val_loss: -31.1428 - val_accuracy: 0.7924
Epoch 2/10
9263/9263 [==============================] - 29s 3ms/step - loss: -45.5892 - accuracy: 0.7924 - val_loss: -59.9430 - val_accuracy: 0.7924
Epoch 3/10
9263/9263 [==============================] - 33s 4ms/step - loss: -74.3504 - accuracy: 0.7924 - val_loss: -88.7384 - val_accuracy: 0.7924
Epoch 4/10
9263/9263 [==============================] - 33s 4ms/step - loss: -103.0489 - accuracy: 0.7924 - val_loss: -117.5095 - val_accuracy: 0.7924
Epoch 5/10
9263/9263 [==============================] - 33s 4ms/step - loss: -131.6506 - accuracy: 0.7924 - val_loss: -146.2309 - val_accuracy: 0.7924
Epoch 6/10
9263/9263 [==============================] - 29s 3ms/step - loss: -160.5589 - accuracy: 0.7924 - val_loss: -175.0475 - val_accuracy: 0.7924
Epoch 7/10
9263/9263 [==============================] - 30s 3ms/step - loss: -189.3223 - accuracy: 0

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred = model.predict(X_test)
classes_x=np.argmax(y_pred,axis=1)
#print(classes_x)

6176/6176 [==============================] - 9s 1ms/step - loss: -290.1868 - accuracy: 0.7924

Loss: -290.19, Accuracy: 79.24%
